# 02. Generate loop censuses, Denmark-wide
## Project: Bicycle node network loop analysis

This notebook generates a loop census for each node from the preprocessed and snapped Denmark-wide network from 01, and calculates/plots basic descriptive statistics for the whole country of Denmark.  
Please select `denmark` as the `study_area` in the `config.yml`.

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2024-01-24  
Last modified: 2025-08-08

## To do

- [ ] Double-check loop/link lengths. For example 3-loop east of Faxe
- [ ] Double-check edge_ids during simplifications
- [ ] Semilogy scale for loop lengths

## Parameters

In [ ]:
%run -i setup_parameters.py
debug = True  # Set to True for extra plots and verbosity

## Functions

In [ ]:
%run -i functions.py

## Load data

In [ ]:
Gnx = nx.empty_graph()
with lzma.open(PATH["data_out"] + "network_preprocessed.xz", "rb") as f:
    G_new = pickle.load(f)
    Gnx = nx.disjoint_union(Gnx, G_new.to_networkx())
if debug:
    print("N: " + str(Gnx.number_of_nodes()), ", L: " + str(Gnx.number_of_edges()))
    for k, v in list(Gnx.nodes(data=True))[:10]:
        print(k, v)
    for u, v in Gnx.edges(list(range(3))):
        print(u, v)

In [ ]:
# just dummy files that must be saved due to copy paste from old code. eventually fix and remove.
nodes_id = list()
nodes_coords = list()

## Loop generation

### Get face loops

The minimum cycle basis is generally not the cycle basis of face loops, see: https://en.wikipedia.org/wiki/Cycle_basis#In_planar_graphs  
Therefore, we can't use https://python.igraph.org/en/latest/api/igraph.GraphBase.html#minimum_cycle_basis here. Instead, we solve the problem geometrically via shapely.

#### Polygonize

In [ ]:
edgegeoms = list(nx.get_edge_attributes(Gnx, "geometry").values())
facepolygons, _, _, _ = shapely.polygonize_full(edgegeoms)
if debug:
    print(edgegeoms[:10])
    p = gpd.GeoSeries(facepolygons)
    p.plot()
    plt.axis("off")

#### Intersect polygons with graph to get face loops

In [ ]:
# Code from: https://github.com/anastassiavybornova/bike-node-planner/blob/main/scripts/script06.py
ns, es = momepy.nx_to_gdf(net=Gnx, points=True, lines=True)

linestrings = (
    es.geometry.copy()
)  # our geopandas.GeoSeries of linestrings representing street network
collection = shapely.GeometryCollection(linestrings.array)  # combine to a single object
noded = shapely.node(collection)  # add missing nodes
polygonized = shapely.polygonize(
    noded.geoms
)  # polygonize based on an array of nodded parts
polygons = gpd.GeoSeries(polygonized.geoms)  # create a GeoSeries from parts

# create geodataframe of loops, where we will save evaluation column
faceloops = gpd.GeoDataFrame(geometry=polygons, crs=es.crs)
if debug:
    print(faceloops.head(5))

In [ ]:
faceloopnodes = []
for fl in faceloops.itertuples():  # loop through each face loop fl
    nsinters = ns.intersection(
        fl.geometry
    )  # intersect all nodes with the fl's geometry
    nsinters = nsinters[~nsinters.is_empty]  # remove empties
    faceloopnodes.append(len(nsinters))  # save number of nodes

In [ ]:
h = plt.hist(faceloopnodes, range(18))
plt.title("Nodes per faceloop")
print(h[1])
print(h[0])

~~Getting all simple loops has not yet been implemented in igraph, see:  
* https://github.com/igraph/igraph/issues/379  
* https://github.com/igraph/igraph/issues/1398  
Some potential progress here, but only for C, not Python:
* https://github.com/igraph/igraph/pull/2181

But they can be XORed through the loop base.~~

Update 2025-06-19: It *has* been implemented now! 🎉  
https://github.com/igraph/python-igraph/releases/tag/0.11.9  
https://python.igraph.org/en/0.11.9/api/igraph.GraphBase.html#simple_cycles  

It has been implemented in networkX though: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.cycles.simple_cycles.html#networkx.algorithms.cycles.simple_cycles

Therefore, we do not use igraph's loop basis, but go ahead with networkX.

### Get all loops via igraph

to do

### Get all loops via nx

In [ ]:
# Get all loops, meaning a loop ABCA is counted also as BCAB and CABC
allloops, alllooplengths, allloopnumnodes, allloopmaxslopes = get_allloops_nx(Gnx)

## Save loop census

In [ ]:
if LOOP_LENGTH_BOUND:
    llb_string = "_maxlength" + str(LOOP_LENGTH_BOUND)
else:
    llb_string = ""

with open(
    PATH["data_out"] + "loopcensus_" + str(LOOP_NUMNODE_BOUND) + llb_string + ".pkl",
    "wb",
) as f:
    pickle.dump(allloops, f)
    pickle.dump(alllooplengths, f)
    pickle.dump(allloopnumnodes, f)
    pickle.dump(allloopmaxslopes, f)
    pickle.dump(Gnx, f)
    pickle.dump(LOOP_NUMNODE_BOUND, f)
    pickle.dump(nodes_id, f)
    pickle.dump(nodes_coords, f)
    pickle.dump(numloops, f)
    pickle.dump(faceloops, f)